In [1]:
from BRScraper import nba

df = nba.get_current_salaries(info='players')
print(df)

                    Player   Tm      2024-25      2025-26      2026-27  \
0            Stephen Curry  GSW  $55,761,216  $59,606,817  $62,587,158   
1              Joel Embiid  PHI  $51,415,938  $55,224,526  $57,985,752   
2             Nikola Jokić  DEN  $51,415,938  $55,224,526  $59,033,114   
3             Kevin Durant  PHO  $51,179,021  $54,708,609          NaN   
4             Bradley Beal  PHO  $50,203,930  $53,666,270  $57,128,610   
..                     ...  ...          ...          ...          ...   
454            Malik Fitts  IND     $555,216          NaN          NaN   
455         Mamadi Diakite  MEM     $464,050     $464,050     $464,050   
456           Didi Louzada  POR     $268,032     $268,032     $268,032   
457             AJ Griffin  HOU     $250,000          NaN          NaN   
458  Javon Freeman-Liberty  TOR     $100,000          NaN          NaN   

         2027-28      2028-29 2029-30    Guaranteed  
0            NaN          NaN     NaN  $177,955,191  
1  